# CV 7

 to install netcat:
    apt update
    apt-get install netcat
    nc -lkp 9999 &

In [9]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col,regexp_replace

spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()



In [76]:
spark.stop()

In [77]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col,regexp_replace,current_timestamp, window, desc

spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()
lines = spark.readStream.option('newFilesOnly', 'true').text('./data')
#lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

words = lines.select(explode(split(lines.value, " ")).alias("word"))

words = words.select(lower(col('word')).alias('word'))
words = words.withColumn('word', regexp_replace('word', r'[^\s\w]', ''))
words = words.filter("word != ''")

words = words.withColumn('eventTime', current_timestamp())
wordCounts = words.groupBy(window(col("eventTime"), "30 seconds", "15 seconds").alias('start'), col("word")).count()
wordSorted = wordCounts.orderBy("start",desc("count"))


query = wordSorted.writeStream.outputMode("complete").format("memory").queryName("my_query").start()

while True:
    #spark.sql("SELECT * FROM my_query").show(truncate=False)
    # just show 1 window each time
    latest_window_query = """
    SELECT *
    FROM my_query
    WHERE start = (SELECT MAX(start) FROM my_query)
    """

    spark.sql(latest_window_query).show(truncate=False)

    time.sleep(5)


23/12/01 12:01:43 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e07249ac-ef24-466b-a321-44cfa6a8c326. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/01 12:01:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----+----+-----+
|start|word|count|
+-----+----+-----+
+-----+----+-----+


+------------------------------------------+------------+-----+
|start                                     |word        |count|
+------------------------------------------+------------+-----+
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|bruh        |21   |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|here        |5    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|hello       |2    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|bruhbruhbruh|1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|dfaasf      |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|aaaaaaaaaa  |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|dsdsdasf    |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|count       |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|to          |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|now         |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|world       |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|hele        |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:

Exception in thread "stream execution thread for my_query [id = 7e860307-f1af-4230-b3ef-5c5453bcd420, runId = 6f04a8d6-14d3-4a76-ab2b-7d23d01b2521]" org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.sql.execution.streaming.state.StateStoreCoordinatorRef.deactivateInstances(StateStoreCoordinator.scala:119)
	at org.apache.spark.sql.streaming.StreamingQueryManager.notifyQueryTermination(StreamingQueryManager.scala:406)
	at org.apache.spark.sql.execution.streaming.StreamExecution.$anonfun$runStream$3(StreamExecution.scala:357)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(U

+------------------------------------------+------------+-----+
|start                                     |word        |count|
+------------------------------------------+------------+-----+
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|bruh        |21   |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|here        |5    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|hello       |2    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|bruhbruhbruh|1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|dfaasf      |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|aaaaaaaaaa  |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|dsdsdasf    |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|count       |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|to          |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|now         |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|world       |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:00}|hele        |1    |
|{2023-12-01 12:01:30, 2023-12-01 12:02:

KeyboardInterrupt: 

# cv 7 bonus

In [72]:
# the same thing as above but this time when a new file is added to the directory, the whole directory is read again including the new file

import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, lower, col,regexp_replace,current_timestamp, window, desc

spark = SparkSession.builder.appName("NetworkWordCount").getOrCreate()

lines = spark.readStream.option('newFilesOnly', 'true').text('./data')

words = lines.select(explode(split(lines.value, " ")).alias("word"))

words = words.select(lower(col('word')).alias('word'))

words = words.withColumn('word', regexp_replace('word', r'[^\s\w]', ''))

words = words.filter("word != ''")

words = words.withColumn('eventTime', current_timestamp())

wordCounts = words.groupBy(window(col("eventTime"), "30 seconds", "15 seconds").alias('start'), col("word")).count()

wordSorted = wordCounts.orderBy("start",desc("count"))

query = wordSorted.writeStream.outputMode("complete").format("memory").queryName("my_query").start()

while True:
    #spark.sql("SELECT * FROM my_query").show(truncate=False)
    # just show 1 window each time
    latest_window_query = """
    SELECT *
    FROM my_query
    WHERE start = (SELECT MAX(start) FROM my_query)
    """

    spark.sql(latest_window_query).show(truncate=False)

    time.sleep(5)

23/12/01 11:59:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-40e17a3b-c4e8-4155-8077-d25fd922dfd4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/01 11:59:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


IllegalArgumentException: Cannot start query with name my_query as a query with that name is already active in this SparkSession